# Utilizando la columna “opinión” del set de datos de opiniones:

In [6]:
import pandas as pd
# Cargar el archivo Excel
file_path = 'Rest_Mex_2022_Sentiment_Analysis_Track_Train.xlsx'
df = pd.read_excel(file_path)

- Normalizar el texto usando spacy

In [7]:
import spacy
import unidecode

# Cargar el modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Función para normalizar el texto usando spaCy
def normalize_text(text):
    # Cast a str
    text = str(text)
    # Convertir a minúsculas
    text = text.lower()
    # Quitar acentos
    text = unidecode.unidecode(text)
    # Procesar el texto con spaCy
    doc = nlp(text)
    # Eliminar stop words y puntuación, y realizar lematización
    text = ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
    return text

- Normalzar las opiniones paralelizado

In [8]:
from concurrent.futures import ThreadPoolExecutor

def parallel_processing_threads(data, func):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(func, data))
    return results

    # Normaliza el dataframe en paralelo
df['Normalized_Opinion'] = parallel_processing_threads(df['Opinion'], normalize_text)

- Entrenar un modelo Doc2vec

In [18]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Preparar los datos
tokenized_opinions = [opinion.split() for opinion in df['Normalized_Opinion'].astype(str)]
tagged_data = [TaggedDocument(words=words, tags=[str(attraction)]) for words, attraction in zip(tokenized_opinions, df['Attraction'].astype(str))] # Se puede hacer para polarity

# Construir y entrenar el modelo
model = Doc2Vec(vector_size=50, window=5, min_count=1, workers=4, epochs=20)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Guardar el modelo para uso futuro
model.save("doc2vec_model")

- Entrenar un algoritmo knn

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from gensim.models import Doc2Vec
import pandas as pd
import numpy as np

# Cargar el modelo Doc2Vec entrenado
model = Doc2Vec.load("doc2vec_model")

# Preparar los datos para el entrenamiento
df['Normalized_Opinion'] = df['Normalized_Opinion'].apply(lambda x: str(x) if pd.notnull(x) else '')
feature_vectors = [model.infer_vector(opinion.split()) for opinion in df['Normalized_Opinion']]
X = np.array(feature_vectors)
y = df['Attraction'].values

# Entrenar el clasificador k-NN
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

- Realizar pruebas con 50 opiniones

In [24]:
# Realizar pruebas con 50 opiniones
X_test_sample = X_test[:50]
y_test_sample = y_test[:50]

# Hacer predicciones
y_pred = knn.predict(X_test_sample)

- Obtener matriz de confusion y acc

In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Generar la Matriz de Confusión y calcular Accuracy
conf_matrix = confusion_matrix(y_test_sample, y_pred)
accuracy = accuracy_score(y_test_sample, y_pred)

print("Matriz de Confusión:")
print(conf_matrix)
print("Accuracy:", accuracy)

Matriz de Confusión:
[[ 4  0  1]
 [ 3 24  2]
 [ 0  3 13]]
Accuracy: 0.82


- Matriz de confusion para el total del conjunto de prueba

In [26]:
# Realizar predicciones en el conjunto de prueba (80% entrenamiento, 20% prueba)
y_pred = knn.predict(X_test)

# Evaluar el modelo: generar la Matriz de Confusión y calcular el Accuracy
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Matriz de Confusión:")
print(conf_matrix)
print("Accuracy:", accuracy)

Matriz de Confusión:
[[ 873   56   90]
 [ 239 2746  312]
 [ 298  347 1082]]
Accuracy: 0.7779248717524408
